# Classifying Suspicious Firms

#### Citation of Data
Hooda,Nishtha. (2018). Audit Data. UCI Machine Learning Repository. https://doi.org/10.24432/C5930Q.

## Introduction 

Having the need to interact with firms on a daily basis, many people in the world have been victims of fraudulent firms. Although auditors may be able to track a few down, not all can be located. Can we use present and historical risk factors to determine if a firm can be labeled as a fraudulent risk or not?

The data, stored as a CSV file, contains one year (2015-2016) exhaustive information on firms that is collected by the Auditor Office of India. The aim of this project is to use this data to develop a model which will help auditors by classifying and finding fraudulent firms. A variety of risk factors are examined in order to reach a score of either 1 (fraudulent) or 0 (not fraudulent) for a firm. There are missing values in the dataset that will be corrected by tidying the data.


## Exploratory data analysis

First, we load in the packages as well as loading in our data.

In [ ]:
set.seed(888)

# Loads in the packages
library(tidyverse)
library(repr)
library(tidymodels)
install.packages("kknn")
library(kknn)


# Loads the data into the variable
url <- "https://raw.githubusercontent.com/alexander-zhong/dsci-100-project/main/data/audit_risk.csv"
raw_audit_risk_data <- read_csv(url) 

head(raw_audit_risk_data)

As we can see from the preview, there is a lot of tidying and cleaning of
the data that needs to be done before using it. In addition, from a quick glance of the data, we can recognize that using these variables (inherent risk, detection risk, audit risk, control risk) will be ideal for our classifier since other variables in this dataset do not have a clear definition or meaning/information behind them making it very hard to believe if they are variables to can contribute at all to our classifier. So, lets tidy the data and select the variables we need.

In [ ]:
# Clean up and turn the Risk variable into a factor for our classification
audit_risk_data <- raw_audit_risk_data |>
    mutate(Risk = as_factor(Risk)) |>
    mutate(Risk = fct_recode(Risk, "Fraudulent" = "1", "Not Fraudulent" = "0")) |>
    select(Risk, CONTROL_RISK, Inherent_Risk, Audit_Risk) |>
    drop_na()
    

head(audit_risk_data)

Lets quickly take a look at our data by representing the data with a few 
tables:

In [ ]:
# see the total number of firms that are considered a risk or not a risk
preliminary_table1 <- audit_risk_data |>
    group_by(Risk) |>
    summarize(Count = n())
preliminary_table1


# see the average of our predictor variables
preliminary_table2 <- audit_risk_data |>
    select(CONTROL_RISK, Audit_Risk, Inherent_Risk) |>
    map_df(mean, na.rm = TRUE)

preliminary_table2

The amount of fraudulent firms is way greater than we expected! Lets take look at the scatter plot distribution of our data with the variables we decided to use. Lets also standardize the data first so we can have a more accurate comparison between the variables.

In [ ]:
set.seed(120)
options(jupyter.plot_mimetypes = c("text/plain", "image/png" ))

preliminary_plot1 <- audit_risk_data |> 
    ggplot(aes(x = Inherent_Risk, y = Audit_Risk, color = Risk)) +
    geom_point(alpha = 0.4) + 
    labs(x = "Inherent Risk (Unstandardized)", y = "Audit Risk (Unstandardized)") +
    ggtitle("Inherent Risk against Audit Risk") +
    scale_x_log10(labels = label_comma()) +
    ylim(0, 10) +
    ggtitle("Inherent Risk against Audit Risk (Scaled with Log base of 10)")

preliminary_plot1




#### Standardizing the data
audit_recipe <- recipe(Risk ~ CONTROL_RISK + Audit_Risk + Inherent_Risk, data = audit_risk_data) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
    prep()

audit_risk_scaled <- bake(audit_recipe, audit_risk_data)

head(audit_risk_scaled)

preliminary_plot2 <- audit_risk_scaled |> 
    ggplot(aes(x = Inherent_Risk, y = Audit_Risk, color = Risk)) +
    geom_point(alpha = 0.2) + 
    labs(x = "Inherent Risk (Standardized)", y = "Audit Risk (Standardized)") +
    ggtitle("Standardized Inherent Risk against Audit Risk") +
    xlim(-0.45, 0.45) +
    ylim(-0.25, 0)
    
preliminary_plot2

From all our visualization and tables, we can conclude that the three variables (control risk, inherent risk, and audit risk) can be used to train our classifier. As we can see, there is a clear upward trend in the scatter plots of Audit Risk vs Inherent Risk leading to more fraudulent firms which further supports our decision in using these variables.

## Building Classifier

### Tuning

This classifier will be based on the K-Nearest Neighbors algorithm to determine
whether a firm is considered fraudulent or not. In order to do so, tune and evaluate the classifier for the most optimal K value and testing the accuracy.

First, lets scale the data as well as split the data randomly with set.seed(888) in code cells above for a 70% training data and a 30% testing data.

In [ ]:
set.seed(120)

audit_risk_split <- initial_split(audit_risk_data, prop = .7, strata = Risk)
audit_risk_train <- training(audit_risk_split)
audit_risk_test <- testing(audit_risk_split)

audit_recipe <- recipe(Risk ~ CONTROL_RISK + Audit_Risk + Inherent_Risk, data = audit_risk_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

Now, we want to use cross validation within our training data to choose the correct k-value.

In [ ]:
set.seed(120)

audit_vfold <- vfold_cv(audit_risk_train, v = 5, strata = Risk) 


knn_spec <- nearest_neighbor(
    weight_func = "rectangular", 
    neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

knn_results <- workflow() |>
    add_recipe(audit_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = audit_vfold, grid = 10) |>
    collect_metrics()



In [ ]:
set.seed(120)

accuracies <- knn_results |> 
       filter(.metric == "accuracy")

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "K Value", y = "Accuracy Estimate") +
    scale_x_continuous(breaks = seq(0, 14, by = 1)) 
accuracy_versus_k


As we can see from our tuning, the value 4 would provide a very high accuracy. But, as we increase the k value from 4, there is a dramatic drop. In our case, we should built our classifier with the k-value of 4 to maximize accuracy.

### Constructing the Classifier with K Value of 4

In [ ]:
audit_recipe <- recipe(Risk ~ CONTROL_RISK + Audit_Risk + Inherent_Risk, data = audit_risk_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 4) |>
       set_engine("kknn") |>
       set_mode("classification")


audit_fit <- workflow() |>
    add_recipe(audit_recipe) |>
    add_model(knn_spec) |>
    fit(data = audit_risk_train)

audit_predictions <- predict(audit_fit, audit_risk_test) |>
    bind_cols(audit_risk_test)

head(audit_predictions)

Now that we have used our training data to creat our classifier and used it to predict the training data, we can take a look at the metrics to see how well our classifier did!

In [ ]:
audit_prediction_metrics <- audit_predictions |>
    metrics(truth = Risk, estimate = .pred_class)


audit_prediction_metrics

As we can see here, our accuracy of our classifier is pretty high with
0.9957265 success rate when classifying our testing data.

## Discussion
- summarize what you found
- discuss whether this is what you expected to find?
- discuss what impact could such findings have?
- discuss what future questions could this lead to?

## References

...